In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some, axes_dict, plot_history
from csbdeep.io import save_tiff_imagej_compatible, load_training_data
from csbdeep.models import CARE, Config
from csbdeep.data import RawData, create_patches, create_patches_reduced_target
import shutil 
import os
# # renomme fichiers pour csbdeep 
# import os

date = "070418a"
path = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/data/train/train_samples"
pathUniform_train = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataUniform/train/train_samples"
pathUniform_test = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataUniform/test/test_samples"

def renomme_data(date, tiffdata_path, demi_pas=False) :
    
    if demi_pas :
        return None 
    
    for file_name in os.listdir(tiffdata_path) :
        
        if file_name[:len(date)] != date or file_name[-4:] != ".tif" :
            continue
        
        # nbre de caractères du nombre 
        s=0
        while file_name[len(date) + 2 + s] != "_" :
            s+= 1
        assert s%2 == 0
        
        
        nbre =  ( int(file_name[len(date)+2:len(date)+2+s//2]) + int(file_name[len(date)+2+s//2:len(date)+2+s]) ) // 2    
        nvnom = file_name[:len(date)+2] + str(nbre).zfill(3) + file_name[len(date)+2+s:]
        
        path_old = tiffdata_path + "/" + file_name
        path_new = tiffdata_path + "/" + nvnom
        print(nvnom, nbre)
        
        #os.rename(path_old, path_new)
        
    return None

# données
dir_data = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/data"
dir_dataUniform = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataUniform" 
dir_dataDemistep = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataDemistep" 
dir_dataMix = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataMix"

def fetch_rawdata(dir_data):
    raw_data = RawData.from_folder (
        basepath = dir_data + "/train",
        source_dirs = [dir_data + "/train/train_samples"],
        target_dir = dir_data + "/train/train_labels",
        axes = "ZCYX" )
    
    return raw_data

#renomme_data(pathUniform_train, demi_pas=False)
#renomme_data(pathUniform_test, demi_pas=False)
rawdata = fetch_rawdata(dir_dataMix)

In [ ]:
save_file_dir = dir_dataMix+"/my_training_data.npz"

# création des patches
X, Y, XY_axes = create_patches_reduced_target(
    reduction_axes      = "C",
    target_axes         = "ZYX",
    raw_data            = rawdata,
    patch_size          = (32,4,64,64),
    n_patches_per_image = 120,
    save_file           = save_file_dir,
)

In [2]:
#dir_dataDemistep = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataDemistep" 
dir_dataMix = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataMix" 
data_path = dir_dataMix+"/my_training_data.npz"
(X,Y), (X_val,Y_val), axes = load_training_data(data_path, validation_split=0.1, 
                                                verbose=True)
c = axes_dict(axes)['C']
n_channel_in, n_channel_out = X.shape[c], Y.shape[c]
config = Config(axes, n_channel_in, n_channel_out, train_batch_size=16)

number of training images:	 21492
number of validation images:	 2388
image size (3D):		 (32, 64, 64)
axes:				 SZYXC
channels in / out:		 4 / 1


In [3]:
ind = 5
model = CARE(config, 'modele_interpolation'+str(ind))

%load_ext tensorboard
%tensorboard --logdir { "C:/Users/bioAMD/Desktop/Nathan" }
history = model.train(X,Y, validation_data = (X_val, Y_val))

savehist_path = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/trainHistoryDict" + str(ind) + ".txt"
epoch = np.arange(start=1, stop=len(history.history["loss"])+1, step=1 )

if os.path.exists(savehist_path) :
    os.remove(savehist_path)
with open(savehist_path, 'w') as file_pi:
    file_pi.write("# epoch loss mse mae val_loss val_mse val_mae \n")
    for i in range(len(history.history["loss"])) :
        file_pi.write(str(epoch[i]) + " ")
        file_pi.write(str(history.history["loss"][i]) + " ")
        file_pi.write(str(history.history["mse"][i]) + " ")
        file_pi.write(str(history.history["mae"][i]) + " ")
        file_pi.write(str(history.history["val_loss"][i]) + " ")
        file_pi.write(str(history.history["val_mse"][i]) + " ")
        file_pi.write(str(history.history["val_mae"][i]) + "\n")

#ind += 1 # pour séparer les history de deux entrainements

C:\Users\bioAMD\anaconda3\envs\care\lib\site-packages\csbdeep\models\base_model.py:149: UserWarning: output path for model already exists, files may be overwritten: C:\Users\bioAMD\Desktop\Nathan\modele_interpolation5
  warnings.warn('output path for model already exists, files may be overwritten: %s' % str(self.logdir.resolve()))


Epoch 1/100
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x0000020ADA4611F0>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names.
Match 0:
lambda x: K.mean(x, axis=-1)

Match 1:
lambda x: x

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x0000020ADA4611F0>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names.
Match 0:
lambda x: K.mean(x, axis=-1)

Match 1:
lambda x: x

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x0000020ADA4613A0>: found multiple defi

Epoch 28/100
400/400 [==============================] - 210s 526ms/step - loss: 0.0496 - mse: 0.0073 - mae: 0.0496 - val_loss: 0.0496 - val_mse: 0.0073 - val_mae: 0.0496
Epoch 29/100
400/400 [==============================] - 215s 538ms/step - loss: 0.0491 - mse: 0.0072 - mae: 0.0491 - val_loss: 0.0495 - val_mse: 0.0072 - val_mae: 0.0495
Epoch 30/100
400/400 [==============================] - 214s 534ms/step - loss: 0.0498 - mse: 0.0073 - mae: 0.0498 - val_loss: 0.0496 - val_mse: 0.0073 - val_mae: 0.0496
Epoch 31/100
400/400 [==============================] - 247s 618ms/step - loss: 0.0492 - mse: 0.0072 - mae: 0.0492 - val_loss: 0.0496 - val_mse: 0.0072 - val_mae: 0.0496
Epoch 32/100
400/400 [==============================] - 228s 570ms/step - loss: 0.0493 - mse: 0.0072 - mae: 0.0493 - val_loss: 0.0495 - val_mse: 0.0073 - val_mae: 0.0495
Epoch 33/100
400/400 [==============================] - 239s 597ms/step - loss: 0.0495 - mse: 0.0072 - mae: 0.0495 - val_loss: 0.0495 - val_mse: 0.007

KeyboardInterrupt: 

In [ ]:
#savehist_path = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/trainHistoryDict" + str(ind) + ".txt"
model = CARE(config, 'modele_interpolation'+str(ind))

savehist_path = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/trainHistoryDict" + str(ind) + ".txt"
epoch = np.arange(start=1, stop=32, step=1 )

if os.path.exists(savehist_path) :
    os.remove(savehist_path)
with open(savehist_path, 'w') as file_pi:
    file_pi.write("# epoch loss mse mae val_loss val_mse val_mae \n")
    for i in range(len(history.history["loss"])) :
        file_pi.write(str(epoch[i]) + " ")
        file_pi.write(str(history.history["loss"][i]) + " ")
        file_pi.write(str(history.history["mse"][i]) + " ")
        file_pi.write(str(history.history["mae"][i]) + " ")
        file_pi.write(str(history.history["val_loss"][i]) + " ")
        file_pi.write(str(history.history["val_mse"][i]) + " ")
        file_pi.write(str(history.history["val_mae"][i]) + "\n")

xy_patchsize = 64
batchsize_max = 7.7*1e3 // ( 32*(xy_patchsize**2) *4* 1e-6)
print(batchsize_max)

#with open('/trainHistoryDict' + str(ind), "rb") as file_pi:
#    history = pickle.load(file_pi)